#### Return the tenth element of a list.

Script variation 1:

In [1]:
(defun our-tenth (x)
    (nth 9 x))

OUR-TENTH

In [2]:
(our-tenth '(1 2 3 4 5 6 7 8 9 10 11))

10

In [4]:
(our-tenth '(a b c d e f g h i j k l m o p q r s \t u v w x z y))

J

Script variation 2:

In [6]:
(defun our-tenth2 (x)
    (car (cdr (cdr (cdr (cdr (cdr (cdr (cdr (cdr (cdr x)))))))))))

OUR-TENTH2

In [7]:
(our-tenth2 '(1 2 3 4 5 6 7 8 9 10 11))

10

In [8]:
(our-tenth2 '(a b c d e f g h i j k l m o p q r s \t u v w x z y))

J

#### Show the number and its cube for the values between 2 and 10.

In [11]:
(defun show-cubes (values)
    (mapcar
     (lambda (x) (format t "~D cubed: ~D~%" x (* x x x)))
     values))

SHOW-CUBES


REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::SHOW-CUBES in DEFUN


In [12]:
(show-cubes '(2 3 4 5 6 7 8 9 10))

(NIL NIL NIL NIL NIL NIL NIL NIL NIL)

2 cubed: 8
3 cubed: 27
4 cubed: 64
5 cubed: 125
6 cubed: 216
7 cubed: 343
8 cubed: 512
9 cubed: 729
10 cubed: 1000


In [18]:
(show-cubes '(4 5 6 7))

(NIL NIL NIL NIL)

4 cubed: 64
5 cubed: 125
6 cubed: 216
7 cubed: 343


#### Implement an informed search and have as an argument eight queens randomly placed on a board and return a list that consists of the same queens in locations that they do not threaten each other. Pick an admissible heuristic of your choice.

Heuristic: Number of queens that are threatened. If it is 0, we have reached our goal state.

In [31]:
(defun make-board (num-queens)
    (setf board (make-array '(8 8))) ;; make empty board
    (dotimes (i num-queens) ;; for each of 8 queens,

        (loop ; loop until we generate a random coordinate that doesn't already have a queen in it
            (setf x (random 8) y (random 8)) 
            (when (not (eql (aref board x y) 1)) 
                (return))
        )

        (setf (aref board x y) 1) ;; add queen to this coordinate
    ) board ;; return board
) 

(defun out-of-bounds (board x y) 
    (or (>= x (array-dimension board 0)) 
        (>= y (array-dimension board 1)) 
        (< y 0) 
        (< x 0)
    )
)

(defun is-queen (board x y)
    (eql (aref board y x) 1)
)

(defun search-row (board x y)
    (let ((num-threats 0)) 
        (dotimes (a (array-dimension board 1)) ;; loop over the columns in that row
            (if (and (not (eql x a)) (is-queen board a y)) ;; if the cell is a queen and not the same queen
                (progn
                    ;(format t "(~A, ~A) is a queen in the same row~%" a y)
                    (incf num-threats 1) ;; immediately return true - there is another queen in this row
                )
            )
        )
         
        num-threats
    )
)

(defun search-column (board x y)
    (let ((num-threats 0))
        (dotimes (b (array-dimension board 0)) ; loop over the rows in that column
            (if (and (not (eql y b)) (is-queen board x b)) ; if the cell contains a queen
                (progn
                    ;(format t "(~A, ~A) is a queen in the same column~%" x b)
                    (incf num-threats 1) ; immediately return true - there is a queen in this row
                )
            )
        )
        num-threats
    )
)

(defun search-up-left-diagonal (board x y)
    
    (let ((num-threats 0))
        ;(format t "Searching up the left diagonal from (~A, ~A)...~%" x y)
        (do ((a (- x 1) (- a 1)) 
             (b (- y 1) (- b 1))) 
            ((out-of-bounds board a b) nil) ; end clause
            (if (is-queen board a b) ;; if current coordinate holds queen
                (progn
                    ;(format t "(~A, ~A) is a queen in the same diagonal~%" a b) ;; print info
                    (incf num-threats 1) ;; return true
                )
            )
        )
        num-threats
    )
)

(defun search-up-right-diagonal (board x y)
    
    (let ((num-threats 0))
        ;(format t "Searching up the right diagonal from (~A, ~A)...~%" x y)
        (do ((a (+ x 1) (+ a 1)) 
             (b (- y 1) (- b 1))) 
            ((out-of-bounds board a b) nil) ; end clause
            (if (is-queen board a b) ;; if current coordinate holds queen
                (progn
                    ;(format t "(~A, ~A) is a queen in the same diagonal~%" a b) ;; print info
                    (incf num-threats 1) ;; return true
                )
            )
        )
        num-threats
    )
)

(defun search-down-left-diagonal (board x y)
    ;(format t "Searching down the left diagonal from (~A, ~A)...~%" x y)
    (let ((num-threats 0))
        (do ((a (- x 1) (- a 1)) 
             (b (+ y 1) (+ b 1))) 
            ((out-of-bounds board a b) nil) ; end clause
            (if (is-queen board a b) ;; if current coordinate holds queen
                (progn
                    ;(format t "(~A, ~A) is a queen in the same diagonal~%" a b) ;; print info
                    (incf num-threats 1) 
                )
            )
        )
        num-threats
    )
)

(defun search-down-right-diagonal (board x y)
    
    (let ((num-threats 0))
        ;(format t "Searching down the right diagonal from (~A, ~A)...~%" x y)
        (do ((a (+ x 1) (+ a 1)) 
             (b (+ y 1) (+ b 1))) 
            ((out-of-bounds board a b) nil) ; end clause
            (if (is-queen board a b) ;; if current coordinate holds queen
                (progn
                    ;(format t "(~A, ~A) is a queen in the same diagonal~%" a b) ;; print info
                    (incf num-threats 1) ;; return true
                )
            )
        )
        num-threats
    )
)

(defun search-diagonals (board x y) 
    (+ (search-up-left-diagonal board x y)
        (search-up-right-diagonal board x y)
        (search-down-left-diagonal board x y)
        (search-down-right-diagonal board x y)
    )
)

(defun get-num-threats (board x y)
    (+ (search-row board x y)
        (search-column board x y)
        (search-diagonals board x y))
)

(defun is-threatened (board x y)
    (if (> (get-num-threats board x y) 0)
        t
        nil
    )
)

(defun get-threatened-queens (board)
    (let ((threatened-queens '()))
        (dotimes (x (array-dimension board 0)) ; loop over columns
            (dotimes (y (array-dimension board 1)) ; loop over rows
                (if (and (is-queen board x y) (is-threatened board x y))
                    (progn
                        ;(format t "(~A, ~A) is a threatened queen~%" x y)
                        (setf threatened-queens (cons (cons x y) threatened-queens))
                    )
                )
            )
        )
         
        threatened-queens ; return total (as last expression in let)
    )
)

(defun enforce-one-queen-per-column (board)
    (dotimes (col (array-dimension board 1)) ; loop over columns (going back to 1st column at the end)
        (let ((has-queen nil)) ; to track whether the column has one queen
            
            (dotimes (row (array-dimension board 0)) ; loop over cells
                (if (eql (aref board row col) 1) ; if cell contains a queen
                    (if (not has-queen) ; if we have no queen in this column yet, track this queen
                        (setf has-queen t)
                        (progn ; otherwise, remove queen
                            (setf (aref board row col) 0)
                        ) 
                    )
                )
            )
             
            (if (not has-queen) ; if this column had no queens, randomly place one
                (setf (aref board (random 8) col) 1)
            )
        )
    )
)

(defun get-random-queen (board)
    (let ((randx (random 8)))
        (dotimes (y (array-dimension board 1))
            (if (is-queen board randx y)
                (return (cons randx y))
            )
        )
    )
)

(defun solve (board max-steps)
    
    (enforce-one-queen-per-column board)
    
    (dotimes (step max-steps) ; repeat for a max amount of times, 
        (if (eql (get-threatened-queens board) 0) ; if we have solved the board, return it
            (progn
                ;(format t "Solved!")
                (return board)
            )
            ;(format t "Threatened queens: ~A~%" (length (get-threatened-queens board)))
        )
            
        (let ((threatened-queens (get-threatened-queens board)) ; get all threatened queens
              (queen nil))
             
             (setf queen (nth (random (length threatened-queens)) threatened-queens)) ; choose random threatened queen
             
             ; find which row in its column where it would be least threatened
             (let ((row-with-min-threats nil) ; (row_index, num_threats set to a high number)
                   (col (car queen))
                   (row (cdr queen)))
                  
                  ;(format t "Dealing with queen at (~A, ~A)...~%" col row)

                  (dotimes (i (array-dimension board 0)) ; for each row, find num of threats
                      (let ((num-threats (get-num-threats board col i)))
                           ;(print (car row-with-min-threats))
                           ;(format t "Checking (~A, ~A)~%" col i)
                           ;(format t "Threatened by ~A queens...~%" num-threats)
                               
                            ; if row-with-min-threats has not yet been initialized
                           ; or if the row's threat is smaller than the tracked row with min threats
                           
                            ; take first row as min threat so far
                            (if (eql row-with-min-threats nil) ; 
                                (setf row-with-min-threats (cons i num-threats))
                                (if (< num-threats (cdr row-with-min-threats)) ; if not first row and current min threats is smaller than tracked min
                                    (setf row-with-min-threats (cons i num-threats))
                                    (if (and (eql num-threats (cdr row-with-min-threats)) 
                                             (eql (random 2) 1))                            ; if current cell's & min cell's threats are equal, we randomly decide which cell to assign
                                        (progn 
                                            ;(format t "Randomly chose (~A, ~A)...~%" col i)
                                            (setf row-with-min-threats (cons i num-threats)))
                                        )
                                    )
                            )
                           
                        )
                 )
                  
                  ;(format t "Least threatened cell is (~A, ~A)...~%" col (car row-with-min-threats))
                  
                  (if (not (eql row (car row-with-min-threats))) ; if its least threatened position isn't where it currently is
                      (progn
                          (setf (aref board (car row-with-min-threats) col) 1) ; move queen
                          (setf (aref board row col) 0) 
                          ;(format t "Moved queen to (~A, ~A)...~%" col (car row-with-min-threats))
                      )
                  )

                  

             )
        )
    )
    
    board
)

MAKE-BOARD

OUT-OF-BOUNDS

IS-QUEEN

SEARCH-ROW

SEARCH-COLUMN

...

GET-THREATENED-QUEENS

ENFORCE-ONE-QUEEN-PER-COLUMN

GET-RANDOM-QUEEN

SOLVE

undefined variable: COMMON-LISP-USER::BOARD
undefined variable: COMMON-LISP-USER::X
undefined variable: COMMON-LISP-USER::Y

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::MAKE-BOARD in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::OUT-OF-BOUNDS in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::IS-QUEEN in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::SEARCH-ROW in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::SEARCH-COLUMN in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::SEARCH-UP-LEFT-DIAGONAL in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::SEARCH-UP-RIGHT-DIAGONAL in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::SEARCH-DOWN-LEFT-DIAGONAL in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::SEARCH-DOWN-RIGHT-DIAGONAL in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining COMMON-LISP-USER::SEARCH-DIAGONALS in DEFUN

REDEFINITION-WITH-DEFUN: 
 

In [25]:
(setf board (make-board 8))

#2A((0 0 0 0 0 0 0 0)
    (0 0 1 0 0 0 0 0)
    (1 0 0 0 0 0 0 0)
    (1 0 0 0 0 0 1 0)
    (0 0 0 0 0 0 1 0)
    (1 0 0 0 0 1 0 0)
    (0 0 0 1 0 0 0 0)
    (0 0 0 0 0 0 0 0))

undefined variable: COMMON-LISP-USER::BOARD


In [32]:
(solve board 100)

#2A((0 0 0 0 0 1 0 0)
    (0 0 1 0 0 0 0 0)
    (1 0 0 0 0 0 0 0)
    (0 0 0 0 0 0 1 0)
    (0 0 0 1 0 0 0 0)
    (0 0 0 0 0 0 0 1)
    (0 0 0 0 1 0 0 0)
    (0 1 0 0 0 0 0 0))